# Testing a trained BIANCA model on Different Images

The purpose of this notebook is to document the process to test a trained model of BIANCA and to have a script that allows re-doing automation of a testing process. After using the BIANCA Model, an analysis is generated on the results, the output of which is in HTML Format.

## This Notebook is only for analysing a Single Timepoint 

### First, we select the files that are to be input by the User as the Flair Image

In [2]:
from tkinter import Tk, filedialog

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
''
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
''
flair_image_path = filedialog.askopenfilename() # Returns opened path as str
print(flair_image_path) 
flair_image_name = flair_image_path.split('/')[-1]
print(flair_image_name)

/Volumes/MS_training/Automated BIANCA Training/Testing Files/ms_013_tp3/ms_013_tp3_flair_brain.nii.gz
ms_013_tp3_flair_brain.nii.gz


### We also ask the user to input a T1 Image

In [3]:
from tkinter import Tk, filedialog

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
''
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
''
t1_image_path = filedialog.askopenfilename() # Returns opened path as str
print(t1_image_path) 
t1_image_name = t1_image_path.split('/')[-1]
print(t1_image_name)

/Volumes/MS_training/Automated BIANCA Training/Testing Files/ms_013_tp3/ms_013_tp3_T1_brain.nii.gz
ms_013_tp3_T1_brain.nii.gz


### File Structure

#### We then create a directory where the analysis is stored

In [4]:
from datetime import date
import os
today = date.today()
directory_name = 'Testing-' + str(today)

In [5]:
# Parent Directory path
x = os.getcwdb()
#print(x)
y = str(x).replace("b'", '')
#print(y)
z = y.replace("'", '')
parent_dir = z
print(parent_dir)

/Users/ms/Desktop/BIANCA_Single_Testing_Notebook


In [6]:
path = os.path.join(parent_dir, directory_name)
try:
    os.mkdir(path)
except OSError as error:
    print(error)

#### Defining the path to store the files in the "Training_{today's date} folder

In [7]:
storage_path = parent_dir + "/" +directory_name

### We also ask for the user to select the 'Training_{date}' folder to extract the dependent information

In [8]:
training_folder_path = filedialog.askdirectory() # Returns opened path as str
print(training_folder_path) 

/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-05-17


## Data Pre-processing

### We perform Brain Extraction on Both the Images

#### For the Storing the Brain Extracted Flair Image

In [9]:
# flair_brain_extracted = "single_timepoint_flair_brain_extracted.nii.gz"
flair_brain_extracted = flair_image_name

#### Defining the directory where the flair image is

In [10]:
directory_path = flair_image_path.replace("/" + flair_image_name, ' ').strip()
directory_path = directory_path.replace(' ', '\ ')

#### Proceeding with the brain extraction

In [19]:
p = os.popen(
    f"cd {directory_path} && cp {flair_image_name} {storage_path}/{flair_brain_extracted}")
if p:
    output = p.read()
    print(output)

#### Repeating the same process for the T1 Brain Image to obtain the brain extracted image

In [17]:
# t1_brain_extracted = "single_timepoint_t1_Brain.nii.gz"
t1_brain_extracted = t1_image_name

In [18]:
p = os.popen(
    f"cd {directory_path} && cp {t1_image_name} {storage_path}/{t1_brain_extracted}")
if p:
    output = p.read()
    print(output)

## Then we Resample the Flair Brain Image into the Dimensions on which our BIANCA Model was trained

#### First we check the dimensions of our Flair Brain Image

In [21]:
def check_dimensions(file_name, printvals):
    p = os.popen(
        f"cd {directory_name} && fslinfo {file_name}")
    if p:
        output = p.read()
        #print(output)
        output = output.replace("\t", "\n")
        file_info = output.split("\n")
        while '' in file_info:
            file_info.remove('')
        #print(file_info)
        dim1 = file_info[3]
        dim2 = file_info[5]
        dim3 = file_info[7]
        if printvals== True:
            print("Dimensions of: ", file_name)
            print(dim1,dim2,dim3)
        return(dim1,dim2,dim3)

In [22]:
flair_dimensions = check_dimensions(flair_brain_extracted, True)

Dimensions of:  ms_013_tp3_flair_brain.nii.gz
256 256 40


#### Next, we check the metadata file that we had stored in the training process

In [23]:
file = open(f"{training_folder_path}/metadata.txt", "r")

# Accessing the metadata file
contents = file.read().split("\n")
dims = contents[0].split(",")
# Retrieving the dimensions
bianca_dimensions = (dims[0], dims[1], dims[2])
print(bianca_dimensions)

('256', '256', '40')


In [24]:
reference_image = contents[1]
print(reference_image)
reference_folder = "_".join(reference_image.split("_")[0:3])
print(reference_folder)

ms_002_tp1_flair_brain.nii.gz
ms_002_tp1


#### Then we check if the dimensions of the selected FLAIR Image are different from the BIANCA Training Set Dimensions. If so, we perform the resampling

We obtain the dimensions of the BIANCA Testing Data

In [25]:
if (flair_dimensions == bianca_dimensions):
    print("Dimensions Match!")
else:
    print("Dimensions do not match. Resampling required!")
    # Resampling the Flair Brain Image
#     The following code bit needs to be modified
#     p = os.popen(
#         f"cd {training_folder_path} && flirt -in {file} -ref {ref_img} -out {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz -omat {directory_name}/{folder_name}/resampling.mat -interp nearestneighbour -dof 6")
#     if p:
#         output = p.read()
#         print(output)

Dimensions Match!


## Then we register our Flair Brain Image onto the MNI Space to obtain the .mat file

#### As a pre-requisite to this step, we need the MNI Space reference file in order to register the resampled flair brains to MNI Space and retrieve the .mat file

#### Since the location of this file is not fixed, we can go ahead and select the file path for the MNI Space file by running the code block below:

In [26]:
from tkinter import Tk, filedialog

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
''
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
''
mni_file = filedialog.askopenfilename() # Returns opened path as str

mni_directory = mni_file.replace(' ', '\ ')
print(mni_directory) 


/Users/ms/Desktop/BIANCA_Single_Testing_Notebook/MNI152_2mm_brain.nii.gz


In [27]:
flair_to_mni = "flair_brain_to_mni.nii.gz"
flair_to_mni_mat = "flair_brain_to_mni.mat"

In [28]:
# Registering the FLAIR Brain Image to MNI Space
p = os.popen(
    f"cd {directory_name} && flirt -in {flair_brain_extracted} -ref {mni_directory} -out {flair_to_mni} -omat {flair_to_mni_mat} -bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7 -interp nearestneighbour")
if p:
    output = p.read()
    print(output)

## Then we register the T1 brain to the Flair Brain to get the t1_to_flair.nii.gz file

In [29]:
t1_to_flair = "t1_brain_to_flair_brain.nii.gz"
t1_to_flair_mat = "t1_brain_to_flair_brain.mat"

In [30]:
# Registering the T1 Image to Flair Image
p = os.popen(
    f"cd {directory_name} && flirt -dof 6 -in {t1_brain_extracted} -ref {flair_brain_extracted} -out {t1_to_flair} -omat {t1_to_flair_mat} -interp nearestneighbour")
if p:
    output = p.read()
    print(output)

## Extracting the Lesion Mask

### Then we Generate the Masterfile

In [31]:
masterfile = f"{directory_name}/masterfile.txt"
flair_lesion_mask = "flair_lesion_mask.nii.gz"

In [33]:
file = open(masterfile, 'w')
file.write(flair_brain_extracted + ' ')
file.write(t1_to_flair + ' ')
file.write(flair_to_mni_mat + ' ')
file.write(flair_lesion_mask + ' ')

25

In [34]:
file.close()

### Then we actually go ahead and use the trained BIANCA Model on our data

#### First, we need to locate the training_labels and mytraining files generated from the testing notebook

In [35]:
classifier = (f"{training_folder_path}/mytraining").replace(" ", "\ ")
print(classifier)
bianca_output = "bianca_output.nii.gz"
binary_bianca_output = "bianca_output_bin.nii.gz"

/Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-05-17/mytraining


In [36]:
print(f"cd {directory_name} && bianca --singlefile={masterfile} --loadclassifierdata={classifier}  --querysubjectnum=1 --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 -o {bianca_output} -v")


cd Testing-2022-05-25 && bianca --singlefile=Testing-2022-05-25/masterfile.txt --loadclassifierdata=/Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-05-17/mytraining  --querysubjectnum=1 --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 -o bianca_output.nii.gz -v


In [37]:
p = os.popen(
    f"cd {directory_name} && bianca --singlefile=masterfile.txt --loadclassifierdata={classifier}  --querysubjectnum=1 --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 -o {bianca_output} -v")
if p:
    output = p.read()
    print(output)

Number of modalities = 4 , number of possible training subjects = 1
Files are: label file list = , data file list = ['masterfile.txt']
Number of training points = 2000, mode = []
Filenames = []
Loading training data  /Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-05-17/mytraining
Training data is size (181166, 5)
Loading query data
  Query file list = ['ms_013_tp3_flair_brain.nii.gz', 't1_brain_to_flair_brain.nii.gz']
 ... reading image ms_013_tp3_flair_brain.nii.gz
 ... reading image t1_brain_to_flair_brain.nii.gz
 ... reading matrix flair_brain_to_mni.mat
Training classifier
Applying classifier
Calculating p-values
Saving output image



/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:170: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pixdims=im.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header m

### Perform the Binary Extraction at a certain Threshold on our obtained BIANCA Output

In [38]:
p = os.popen(
    f"cd {directory_name} && fslmaths {bianca_output} -thr 0.9 -bin {binary_bianca_output}")
if p:
    output = p.read()
    print(output)

## Once the Bianca Output result has been obtained, we move forward and generate a report on the analysis

In [39]:
# As per https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BIANCA/Userguide#Performance_evaluation, 
# The command to run is:

# bianca_cluster_stats <bianca_output_map> <threshold> <min_cluster_size> [<mask>]

p = os.popen(
    f"cd {directory_name} && bianca_cluster_stats {bianca_output} 0.9 1")
if p:
    output = p.read()
    print(output)

 total WMH number for bianca_output.nii.gz with threshold 0.9 and minimum cluster size 1 is 205
 total volume for bianca_output.nii.gz with threshold 0.9 and minimum cluster size 1 is 6373.571777



#### Extracting the number and volume of lesions

In [40]:
num_lesions = output.split("\n")[0].split(" ")[-1]
print(num_lesions)
lesions_vol = output.split("\n")[1].split(" ")[-1]
print(lesions_vol)

205
6373.571777


## Generating the Report

An HTML file by the name of results.html should be generated, which would contain the number of lesions and the total volume of those lesions

In [42]:
file2 = open(f"{directory_name}/results.html", "w")
file2.write("<!DOCTYPE html>\n")
file2.write('<html lang="en" dir="ltr">')
file2.write("<head>")
file2.write("<title>BIANCA RESULTS</title>")
file2.write("</head>")
file2.write("<body>")
file2.write("<h2>BIANCA Results Are:</h2>\n")
file2.write(f"<p>Number of Lesions: {num_lesions}</p>\n")
file2.write(f"<p>Volume of Lesions: {lesions_vol}</p>\n")
file2.write("</body>")
file2.write("</head>")

7

## If the above code block was successful in running, then it means that a test of BIANCA was fun successfully

## Please Ignore the Trial Working below:

In [ ]:
# !pip install -q flask==0.12.2
# !pip install -q flask-ngrok

In [ ]:
# !pip install -q pyngrok
# !ngrok authtoken 22FhSmeHSUGwYAfPE1MOegxwS9Z_2xW7bjhtziVjrDS3tC7nT

In [71]:
# !pip install Jinja2

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [66]:
# folder_name = f"{directory_name}/templates"
# print(folder_name)

Testing-2022-04-25/templates


In [69]:
# !mkdir templates

In [73]:
# ADD HTML CODE HERE
%%writefile templates/Portfolio.html
<!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
  <title>{{name}}'s Portfolio Website</title>
  <style>
    table{
      border:3px solid black;
    }
    th, td {
      border:1px solid black;
    }
    .table{
      width: 100%;
      text-align: center;
    }
</style>
</head>

<body>

    <h1>Welcome to my website!</h1>

</body>
</html>

SyntaxError: invalid syntax (<ipython-input-73-c1f28239281e>, line 3)

In [ ]:
# from flask import Flask, render_template, request
# from flask_ngrok import run_with_ngrok
 
# app = Flask(__name__, template_folder = '/content/templates')
# run_with_ngrok(app)  # Start ngrok when app is run

# @app.route("/")
# def home():
#     your_name = "Zunair Viqar"
#     your_occupation = "Junior at NYU"
#     your_description = "My declared major is in Computer Science with minors in Applied Mathematics and Interactive Media."
#     your_githubUrl = "https://github.com/Zunairviqar"
#     your_favImgDescription = "Here is my favorite picture that I can relate to in Exam Season"
#     projects_list = [
#       {"project": "Graphical User Interface", "purpose" : "For Automated Detection of Lesions in Multiple Sclerosis", "month_completed": "August 2021"},
#       {"project": "enetwa.com", "purpose" : "A website to help out my friend kickstart his e-commerce business", "month_completed": "August 2018 - May 2019"},
#       {"project": "Slack Bot", "purpose" : "Runs daily to obtain data from Google Cloud and posts it on Slack", "month_completed": "July 2021"}
#     ]
#     experiences_list = [
#       {"position": "Member", "institution" : "NYU Abu Dhabi Student Government", "duration": "January 2020 - Present"},
#       {"position": "Student Body President", "institution" : "Nixor College", "duration": "January 2020 - Present"},
#       {"position": "Chief Executive Officer", "institution" : "Nixor Hospital - a student run NGO", "duration": "February 2018 - August 2018"}
#     ]
#     your_favImgUrl = "https://worldwideexperience.com/wp-content/uploads/2020/06/Backup-for-original-945x385.jpg"
#     return render_template("Portfolio.html",name=your_name, 
#                            occupation=your_occupation,
#                            description=your_description, 
#                            githubUrl = your_githubUrl,
#                            projects=projects_list, 
#                            experiences = experiences_list,
#                            favImgDescription = your_favImgDescription,
#                            favImgUrl = your_favImgUrl)